In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install IPyPlot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome("chromedriver", options=chrome_options)
highlight_site=[]
for i in range(3,7):
  baseUrl = 'https://sports.daum.net/schedule/kbo?date=20210' + str(i)
  
  driver.get(baseUrl)
  soup = BeautifulSoup(driver.page_source, "html")
  a_list=soup.find_all('a','link_game link_highlight')
  for href in a_list:
    highlight_site.append('https://sports.daum.net' + href['href'])

In [ ]:
full_video=[]
for site in highlight_site:
  driver = webdriver.Chrome("chromedriver", options=chrome_options)
  driver.get(site)
  soup = BeautifulSoup(driver.page_source, "html")
  a_list=soup.find_all('div','item_thumb')
  for fv in a_list:
    if fv.find('em')==None:
      continue
    if fv.find('em').get_text()=='풀영상':
      full_video.append([fv.find('a','link_tit').get_text(),'https://sports.daum.net/game/80045435/highlight/vod:' + fv['data-id']])

In [ ]:
from IPython.core.display import display, HTML

display(HTML(driver.page_source))

In [ ]:
import ipyplot
import urllib.request
import pandas as pd

url='https://b01-kr-naver-vod.pstatic.net/navertv/c/read/v2/VOD_ALPHA/navertv_2021_07_11_154/5f64b937-e241-11eb-8fc6-246e96398ca5.mp4?_lsu_sa_=6c55b3ff91f062b601de95006525e7b5ee8033e8610b0fe73c77f1cc87ed38858f217aa762b5780d10603252dd39db80029693b703f96164c61b369296ab9ce61c8663f32bad25992a94f12c1798ad93'
urllib.request.urlretrieve(url,'./test.mp4')